# Testing EUGENE Models

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing out EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

In [2]:
import numpy as np
import pandas as pd
import torch

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import claim.modules as cm

# Load data

In [5]:
x = torch.randn(10, 4, 66)
x_t = x.transpose(2, 1)
x_rev = torch.randn(10, 4, 66)
x_rev_t = x_rev.transpose(2, 1)

# FCN Only

In [1]:
from eugene.models.fcn import FCN

In [3]:
fcn = FCN(input_len=66, 
          strand="ss",
          task="binary_classification",
          fc_kwargs=dict(hidden_dims=[100], output_dim=1))
fcn

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


FCN(
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=264, out_features=100, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=100, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [6]:
out = fcn(x, x_rev)
out.shape

torch.Size([10, 1])

In [7]:
out

tensor([[-0.0612],
        [-0.0041],
        [-0.1558],
        [-0.0023],
        [-0.2122],
        [ 0.1105],
        [-0.2584],
        [-0.2296],
        [ 0.1138],
        [ 0.0479]], grad_fn=<AddmmBackward0>)

# CNN Only

In [83]:
from eugene.models.CNN import CNN

<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of a CNN model</b>
</div>

In [86]:
cnn = CNN(input_len=66, 
          strand="ss",
          task="binary_classification", 
          conv_kwargs=dict(channels=[4, 16, 32], conv_kernels=[15, 5, 5], pool_kernels=[1, 1], pool_strides = [1, 1], dropout_rates=0.2), 
          fc_kwargs=dict(output_dim=1))
cnn

CNN(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=1536, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [54]:
out = cnn(x, x_rev)
out.shape

torch.Size([10, 1])

# RNN Only

In [88]:
from eugene.models.RNN import RNN

<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of a RNN model</b>
</div>

In [89]:
rnn = RNN(input_len=66, 
          strand="ss",
          task="regression",
          rnn_kwargs=dict(output_dim=32, unit_type="lstm", bidirectional=True, batch_first=True), 
          fc_kwargs=dict(output_dim=1))
rnn

RNN(
  (rnn): BasicRecurrent(
    (module): LSTM(4, 32, batch_first=True, bidirectional=True)
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (r_squared): R2Score()
)

In [72]:
out = rnn(x_t, x_rev_t)
out.shape

torch.Size([10, 1])

# Hybrid CNN-RNN

In [73]:
from eugene.models.hybrid import hybrid

<div class="alert alert-info" role="alert">
  <b>Just test out the basic function of a CNN model</b>
</div>

In [81]:
cnn_rnn = hybrid(input_len=66, 
                 strand="ts",
                 task="binary_classification", 
                 conv_kwargs=dict(channels=[4, 16, 32], conv_kernels=[15, 5], pool_kernels=[1, 1, 1], pool_strides = [1, 1], dropout_rates=0.2), 
                 rnn_kwargs=dict(output_dim=32, unit_type="lstm", bidirectional=True, batch_first=True),  
                 fc_kwargs=dict(output_dim=1))
cnn_rnn

hybrid(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (reverse_convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (r

In [82]:
out = cnn_rnn(x, x_rev)
out.shape

torch.Size([10, 1])

# Scratch